In [8]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [9]:
#reading the dataset
zomato_real = pd.read_csv("data/zomato.csv", encoding="ISO-8859-1")
zomato_real.head() # prints the first N rows of a DataFrame

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [11]:
zomato_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [12]:
zomato_real['Country Code'].unique()

array([162,  30, 216,  14,  37, 184, 214,   1,  94, 148, 215, 166, 189,
       191, 208])

In [22]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['Has Table booking','Has Online delivery','Is delivering now', 'Switch to order menu', 'Rating color', 'Locality Verbose','Rating text'],axis=1) #Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"


In [23]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [24]:
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)
zomato.info() #.info() function is used to get a concise summary of the dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9542 entries, 0 to 9550
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9542 non-null   int64  
 1   Restaurant Name       9542 non-null   object 
 2   Country Code          9542 non-null   int64  
 3   City                  9542 non-null   object 
 4   Address               9542 non-null   object 
 5   Locality              9542 non-null   object 
 6   Longitude             9542 non-null   float64
 7   Latitude              9542 non-null   float64
 8   Cuisines              9542 non-null   object 
 9   Average Cost for two  9542 non-null   int64  
 10  Currency              9542 non-null   object 
 11  Price range           9542 non-null   int64  
 12  Aggregate rating      9542 non-null   float64
 13  Votes                 9542 non-null   int64  
dtypes: float64(3), int64(5), object(6)
memory usage: 1.1+ MB


In [41]:
#Reading Column Names
zomato.columns

Index(['id', 'restaurant_name', 'country_code', 'city', 'address', 'locality',
       'longitude', 'latitude', 'cuisines', 'cost', 'currency', 'price',
       'rating', 'votes'],
      dtype='object')

In [53]:
#Changing the column names
zomato = zomato.rename(columns={'Restaurant ID': 'id','restaurant_name':'name','Country Code': 'country_code','Average Cost for two':'cost','Price range':'price','rating':'rate'})
zomato.columns

Index(['id', 'name', 'country_code', 'city', 'address', 'locality',
       'longitude', 'latitude', 'cuisines', 'cost', 'currency', 'price',
       'rate', 'votes'],
      dtype='object')

In [54]:
zomato.columns = zomato.columns.str.lower()


In [55]:
zomato.columns

Index(['id', 'name', 'country_code', 'city', 'address', 'locality',
       'longitude', 'latitude', 'cuisines', 'cost', 'currency', 'price',
       'rate', 'votes'],
      dtype='object')

In [56]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float) # Changing the cost to Float
zomato.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9542 entries, 0 to 9550
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9542 non-null   int64  
 1   name          9542 non-null   object 
 2   country_code  9542 non-null   int64  
 3   city          9542 non-null   object 
 4   address       9542 non-null   object 
 5   locality      9542 non-null   object 
 6   longitude     9542 non-null   float64
 7   latitude      9542 non-null   float64
 8   cuisines      9542 non-null   object 
 9   cost          9542 non-null   float64
 10  currency      9542 non-null   object 
 11  price         9542 non-null   int64  
 12  rate          9542 non-null   float64
 13  votes         9542 non-null   int64  
dtypes: float64(4), int64(4), object(6)
memory usage: 1.1+ MB


In [57]:
zomato.head()

,id,name,country_code,city,address,locality,longitude,latitude,cuisines,cost,currency,price,rate,votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100.0,Botswana Pula(P),3,4.8,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City",121.014101,14.553708,Japanese,1200.0,Botswana Pula(P),3,4.5,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,Botswana Pula(P),4,4.4,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City",121.056475,14.585318,"Japanese, Sushi",1500.0,Botswana Pula(P),4,4.9,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City",121.057508,14.584450,"Japanese, Korean",1500.0,Botswana Pula(P),4,4.8,229


In [58]:
#Reading Rate of dataset
zomato['rate'].unique()

array([4.8, 4.5, 4.4, 4.9, 4. , 4.2, 4.3, 3.6, 4.7, 3. , 3.8, 3.7, 3.2,
       3.1, 0. , 4.1, 3.3, 4.6, 3.9, 3.4, 3.5, 2.2, 2.9, 2.4, 2.6, 2.8,
       2.1, 2.7, 2.5, 1.8, 2. , 2.3, 1.9])

In [59]:
zomato['city'].unique()

array(['Makati City', 'Mandaluyong City', 'Pasay City', 'Pasig City',
       'Quezon City', 'San Juan City', 'Santa Rosa', 'Tagaytay City',
       'Taguig City', 'Brasí_lia', 'Rio de Janeiro', 'Sí£o Paulo',
       'Albany', 'Armidale', 'Athens', 'Augusta', 'Balingup',
       'Beechworth', 'Boise', 'Cedar Rapids/Iowa City', 'Chatham-Kent',
       'Clatskanie', 'Cochrane', 'Columbus', 'Consort', 'Dalton',
       'Davenport', 'Des Moines', 'Dicky Beach', 'Dubuque',
       'East Ballina', 'Fernley', 'Flaxton', 'Forrest', 'Gainesville',
       'Hepburn Springs', 'Huskisson', 'Inverloch', 'Lakes Entrance',
       'Lakeview', 'Lincoln', 'Lorn', 'Macedon', 'Macon', 'Mayfield',
       'Mc Millan', 'Middleton Beach', 'Monroe', 'Montville',
       'Ojo Caliente', 'Orlando', 'Palm Cove', 'Paynesville', 'Penola',
       'Pensacola', 'Phillip Island', 'Pocatello', 'Potrero', 'Princeton',
       'Rest of Hawaii', 'Savannah', 'Singapore', 'Sioux City',
       'Tampa Bay', 'Tanunda', 'Trentham East', '

In [60]:
zomato.isnull().sum()

id              0
name            0
country_code    0
city            0
address         0
locality        0
longitude       0
latitude        0
cuisines        0
cost            0
currency        0
price           0
rate            0
votes           0
dtype: int64

In [61]:
zomato.head()

,id,name,country_code,city,address,locality,longitude,latitude,cuisines,cost,currency,price,rate,votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100.0,Botswana Pula(P),3,4.8,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City",121.014101,14.553708,Japanese,1200.0,Botswana Pula(P),3,4.5,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,Botswana Pula(P),4,4.4,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City",121.056475,14.585318,"Japanese, Sushi",1500.0,Botswana Pula(P),4,4.9,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City",121.057508,14.584450,"Japanese, Korean",1500.0,Botswana Pula(P),4,4.8,229


In [62]:
## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()
    

In [63]:
zomato.head()

,id,name,country_code,city,address,locality,longitude,latitude,cuisines,cost,currency,price,rate,votes,Mean Rating
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100.0,Botswana Pula(P),3,4.8,314,4.8
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City",121.014101,14.553708,Japanese,1200.0,Botswana Pula(P),3,4.5,591,4.5
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,Botswana Pula(P),4,4.4,270,4.4
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City",121.056475,14.585318,"Japanese, Sushi",1500.0,Botswana Pula(P),4,4.9,365,4.9
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City",121.057508,14.584450,"Japanese, Korean",1500.0,Botswana Pula(P),4,4.8,229,4.8


In [64]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (1,5))

zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

zomato.sample(3)

,id,name,country_code,city,address,locality,longitude,latitude,cuisines,cost,currency,price,rate,votes,Mean Rating
356,17057591,Bahama Breeze Island Grille,216,Orlando,"8849 International Drive, Orlando, FL 32819",I-Drive/Universal,-81.471526,28.437065,Caribbean,45.0,Dollar($),3,4.3,910,4.51
4954,584,Nathu's Sweets,1,New Delhi,"F-17, Vijay Block, Laxmi Nagar, New Delhi",Laxmi Nagar,77.283502,28.634547,"North Indian, South Indian, Street Food, Chine...",400.0,Indian Rupees(Rs.),1,2.1,121,3.22
3150,5462,Taj Snacks,1,New Delhi,"3575, Netaji Subhash Marg, Daryaganj, New Delhi",Daryaganj,77.240470,28.642815,South Indian,200.0,Indian Rupees(Rs.),1,3.0,5,3.45


In [65]:
zomato.head()

,id,name,country_code,city,address,locality,longitude,latitude,cuisines,cost,currency,price,rate,votes,Mean Rating
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100.0,Botswana Pula(P),3,4.8,314,4.92
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City",121.014101,14.553708,Japanese,1200.0,Botswana Pula(P),3,4.5,591,4.67
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000.0,Botswana Pula(P),4,4.4,270,4.59
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City",121.056475,14.585318,"Japanese, Sushi",1500.0,Botswana Pula(P),4,4.9,365,5.00
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City",121.057508,14.584450,"Japanese, Korean",1500.0,Botswana Pula(P),4,4.8,229,4.92


## Text Preprocessing

* Lower casing
* Removal of Punctuations
* Removal of Stopwords
* Removal of URLs
* Spelling correction

In [66]:
# 5 examples of these columns before text processing:
zomato[['rate', 'cuisines']].sample(5)

,rate,cuisines
1462,2.7,"North Indian, Italian, Mexican"
626,3.6,North Indian
1550,2.4,"North Indian, Chinese"
4642,3.6,"American, Italian, North Indian, Chinese, Leba..."
3741,3.5,Desserts


In [68]:
zomato['cuisines'] = zomato['cuisines'].str.lower()
zomato[['rate', 'cuisines']].sample(5)

,rate,cuisines
7952,0.0,"north indian, chinese, mughlai"
8021,0.0,biryani
1282,3.2,cafe
1833,3.3,chinese
6363,3.6,cafe


In [69]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
restaurant_names

['Le Petit Souffle',
 'Izakaya Kikufuji',
 'Heat - Edsa Shangri-La',
 'Ooma',
 'Sambo Kojin',
 'Din Tai Fung',
 'Buffet 101',
 'Vikings',
 'Spiral - Sofitel Philippine Plaza Manila',
 'Locavore',
 'Silantro Fil-Mex',
 "Mad Mark's Creamery & Good Eats",
 "Guevarra's",
 'Sodam Korean Restaurant',
 'Cafe Arabelle',
 "Nonna's Pasta & Pizzeria",
 'Balay Dako',
 'Hobing Korean Dessert Cafe',
 'Wildflour Cafe + Bakery',
 'NIU by Vikings',
 'The Food Hall by Todd English',
 'Chez Michou',
 'Cafí© Daniel Briand',
 'Casa do Biscoito Mineiro',
 'Maori',
 'Pizza íæ Bessa',
 'Sushi Loko',
 'Beirute',
 'New Koto',
 'Sandubas Cafí©',
 'Villa Tevere',
 'Rovereto',
 'Buena Carne',
 'Taco Pep',
 'Coco Bambu',
 'Taypíç',
 'Outback Steakhouse',
 'Manzuíç',
 'Gero',
 'Brazilian American Burgers',
 'Pesqueiro Eco Gourmet',
 'Confeitaria Colombo',
 'Bibi',
 'Cervantes',
 'Amir',
 'TT Burger',
 'Braseiro da Gíçvea',
 'Balada Mix',
 'Garota de Ipanema',
 'Zazíç BistríÇ Tropical',
 'Filí© de Ouro',
 'D.O.C Rist

In [71]:
zomato.shape

(9542, 15)

In [73]:
zomato.columns

Index(['id', 'name', 'country_code', 'city', 'address', 'locality',
       'longitude', 'latitude', 'cuisines', 'cost', 'currency', 'price',
       'rate', 'votes', 'Mean Rating'],
      dtype='object')

In [74]:
import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [76]:
df_percent.shape

(4771, 15)

In [77]:
df_percent.set_index('name', inplace=True)

In [79]:
indices = pd.Series(df_percent.index)